In [19]:
import numpy as np
import torch as T

### Importujemy potrzebne biblioteki

### autorzy: Adrian Bloch i Witold Jagiełło

# banknoty BNN - klasyfikacja autentyczności

In [20]:
device = T.device("cpu")

### Wykorzystujemy CPU jako domyślny procesor do obliczeń

In [21]:
class BanknoteDataset(T.utils.data.Dataset):

    def __init__(self, src_file, num_rows=None):
        all_data = np.loadtxt(src_file, max_rows=num_rows,
                              usecols=range(1, 6), delimiter="\t", skiprows=0,
                              dtype=np.float32)

        self.x_data = T.tensor(all_data[:, 0:4],
                               dtype=T.float32).to(device)
        self.y_data = T.tensor(all_data[:, 4],
                               dtype=T.float32).to(device)
        self.y_data = self.y_data.reshape(-1, 1)

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        if T.is_tensor(idx):
            idx = idx.tolist()
        preds = self.x_data[idx, :]
        lbl = self.y_data[idx, :]
        sample = {'predictors': preds, 'target': lbl}

        return sample

### ładujemy dane z pliku potem zamieniamy je na tensory PyTorch

In [22]:
def accuracy(model, ds):

    n_correct = 0
    n_wrong = 0


    for i in range(len(ds)):
        inpts = ds[i]['predictors']
        target = ds[i]['target']
        with T.no_grad():
            oupt = model(inpts)

        if target < 0.5 and oupt < 0.5:
            n_correct += 1
        elif target >= 0.5 and oupt >= 0.5:
            n_correct += 1
        else:
            n_wrong += 1

    return (n_correct * 1.0) / (n_correct + n_wrong)


### ds : iterowalnym zestawem danych tensorów
### Tworzy a następnie wylicza DeadLoader

In [23]:
def acc_coarse(model, ds):
    inpts = ds[:]['predictors']
    targets = ds[:]['target']
    with T.no_grad():
        oupts = model(inpts)
    pred_y = oupts >= 0.5
    num_correct = T.sum(targets == pred_y)
    acc = (num_correct.item() * 1.0 / len(ds))
    return acc

### wszystkie rzędy, target 0s i 1s

In [24]:
def my_bce(model, batch):
    sum = 0.0
    inpts = batch['predictors']
    targets = batch['target']
    with T.no_grad():
        oupts = model(inpts)
    for i in range(len(inpts)):
        oupt = oupts[i]

        if targets[i] >= 0.5:
            sum += T.log(oupt)
        else:
            sum += T.log(1 - oupt)

    return -sum / len(inpts)

 ### output powinien uważać na log(0), który oznacza nieskonczoność

In [25]:
class Net(T.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hid1 = T.nn.Linear(4, 8)  # 4-(8-8)-1
        self.hid2 = T.nn.Linear(8, 8)
        self.oupt = T.nn.Linear(8, 1)

        T.nn.init.xavier_uniform_(self.hid1.weight)
        T.nn.init.zeros_(self.hid1.bias)
        T.nn.init.xavier_uniform_(self.hid2.weight)
        T.nn.init.zeros_(self.hid2.bias)
        T.nn.init.xavier_uniform_(self.oupt.weight)
        T.nn.init.zeros_(self.oupt.bias)

    def forward(self, x):
        z = T.tanh(self.hid1(x))
        z = T.tanh(self.hid2(z))
        z = T.sigmoid(self.oupt(z))
        return z

### tworzymy klasę Net korzystającą z nn.Module

In [26]:
def main():
    print("\nBanknote authentication using PyTorch \n")
    T.manual_seed(1)
    np.random.seed(1)


    print("Creating Banknote train and test DataLoader ")

    train_file = ".\\Data\\banknote_k20_train.txt"
    test_file = ".\\Data\\banknote_k20_test.txt"

    train_ds = BanknoteDataset(train_file)  # all rows
    test_ds = BanknoteDataset(test_file)

    bat_size = 10
    train_ldr = T.utils.data.DataLoader(train_ds,
                                        batch_size=bat_size, shuffle=True)


    print("Creating 4-(8-8)-1 binary NN classifier ")
    net = Net().to(device)

    """Trening"""
    print("\nPreparing training")
    net = net.train()  # set training mode
    lrn_rate = 0.01
    loss_obj = T.nn.BCELoss()  # binary cross entropy
    optimizer = T.optim.SGD(net.parameters(),
                            lr=lrn_rate)
    max_epochs = 100
    ep_log_interval = 10
    print("Loss function: " + str(loss_obj))
    print("Optimizer: SGD")
    print("Learn rate: 0.01")
    print("Batch size: 10")
    print("Max epochs: " + str(max_epochs))

    print("\nStarting training")
    for epoch in range(0, max_epochs):
        epoch_loss = 0.0  # for one full epoch
        epoch_loss_custom = 0.0
        num_lines_read = 0

        for (batch_idx, batch) in enumerate(train_ldr):
            X = batch['predictors']  # [10,4]  inputs
            Y = batch['target']  # [10,1]  targets
            oupt = net(X)  # [10,1]  computeds

            loss_val = loss_obj(oupt, Y)  # a tensor
            epoch_loss += loss_val.item()  # accumulate

            optimizer.zero_grad()  # reset all gradients
            loss_val.backward()  # compute all gradients
            optimizer.step()  # update all weights

        if epoch % ep_log_interval == 0:
            print("epoch = %4d   loss = %0.4f" % \
                  (epoch, epoch_loss))

    print("Done ")

    net = net.eval()
    acc_train = accuracy(net, train_ds)
    print("\nAccuracy on train data = %0.2f%%" % \
          (acc_train * 100))
    acc_test = accuracy(net, test_ds)
    print("Accuracy on test data = %0.2f%%" % \
          (acc_test * 100))


    print("\nSaving trained model state_dict \n")
    path = "models.pth"
    T.save(net.state_dict(), path)


    raw_inpt = np.array([[4.4, 1.8, -5.6, 3.2]],
                        dtype=np.float32)
    norm_inpt = raw_inpt / 20
    unknown = T.tensor(norm_inpt,
                       dtype=T.float32).to(device)

    print("Setting normalized inputs to:")
    for x in unknown[0]:
        print("%0.3f " % x, end="")

    net = net.eval()
    with T.no_grad():
        raw_out = net(unknown)  # a Tensor
    pred_prob = raw_out.item()  # scalar, [0.0, 1.0]

    print("\nPrediction prob = %0.6f " % pred_prob)
    if pred_prob < 0.5:
        print("Prediction = authentic")
    else:
        print("Prediction = forgery")

    print("\nEnd Banknote demo")


### tworzymy obiekty Dataset i DataLoader
### wskazujemy pliki do testów i trenowania
### tworzymy sieć nieuronową i wskazujemy jak działa trening
### oceniamy model, zapisujemy go i wykonujemy przewidywanie
###

In [27]:
if __name__ == "__main__":
    main()


Banknote authentication using PyTorch 

Creating Banknote train and test DataLoader 
Creating 4-(8-8)-1 binary NN classifier 

Preparing training
Loss function: BCELoss()
Optimizer: SGD
Learn rate: 0.01
Batch size: 10
Max epochs: 100

Starting training
epoch =    0   loss = 77.0978
epoch =   10   loss = 41.6841
epoch =   20   loss = 14.1492
epoch =   30   loss = 7.9745
epoch =   40   loss = 5.9164
epoch =   50   loss = 4.8247
epoch =   60   loss = 4.3570
epoch =   70   loss = 3.9109
epoch =   80   loss = 3.7746
epoch =   90   loss = 3.6212
Done 

Accuracy on train data = 99.09%
Accuracy on test data = 99.27%

Saving trained model state_dict 

Setting normalized inputs to:
0.220 0.090 -0.280 0.160 
Prediction prob = 0.291901 
Prediction = authentic

End Banknote demo


### startujemy